# Aula 02 - Criando tabela Delta

ACID representa um acrônimo para as quatro características fundamentais que delineiam uma transação: <strong>Atomicidade, Consistência, Isolamento e Durabilidade</strong>. Se uma operação no banco de dados incorpora essas propriedades ACID, ela é classificada como uma transação ACID.

<br/>
<li><strong>Atomicidade</strong> assegura que cada instrução de uma transação, seja leitura, gravação, atualização ou exclusão de dados, seja tratada como uma unidade única. Isso significa que todas as instruções são executadas integralmente ou nenhuma delas é executada. Tal propriedade impede a perda ou corrupção de dados, como quando a fonte de dados de streaming falha no meio do fluxo.</li>

<li><strong>Consistência</strong> garante que as transações modifiquem as tabelas de maneiras predefinidas e previsíveis. Essa consistência transacional mantém as tabelas em um estado consistente mesmo em situações de corrupção de dados ou erros, evitando resultados de execução não intencionais.</li>

<li><strong>Isolamento</strong> permite que vários usuários leiam e gravem na mesma tabela simultaneamente, mas as transações são isoladas, de modo que operações simultâneas não interferem umas nas outras. Na prática, cada solicitação é tratada como se estivesse ocorrendo de forma independente, mesmo que ocorram simultaneamente.</li>

<li><strong>Durabilidade</strong> garante que as alterações de dados provenientes de transações bem-sucedidas sejam preservadas, mesmo em casos de falha do sistema. Essa característica assegura que os dados permaneçam consistentes e acessíveis, mesmo após eventos inesperados ou falhas no sistema.</li>

In [0]:
dataset_uri = 'dbfs:/databricks-datasets/'
display(dbutils.fs.ls(dataset_uri))

#### Olhando dentro do DataSet

In [0]:
dataset_airlines = 'dbfs:/databricks-datasets/airlines/'
display(dbutils.fs.ls(dataset_airlines))

#### Lendo uma fonte de dados

In [0]:
airlines_uri = 'dbfs:/databricks-datasets/airlines/part-00000'
df_airlines = (spark.read
               .load(airlines_uri, 
                    format="csv", 
                    sep=',', 
                    inferSchema="true", 
                    header="true")
              )
df_airlines.limit(5).show()

#### Lendo os dados

In [0]:
df_airlines.limit(5).display()

In [0]:
df_airlines.printSchema()

#### Contando os dados

In [0]:
df_airlines.count() #645.918

#### Create Schema (Database)

#### Salvando os dados em delta

In [0]:
db_name = 'airline'
table_name = 'airline_part_00000'
query_create_schema = f'CREATE SCHEMA IF NOT EXISTS {db_name}'

spark.sql(query_create_schema)
(df_airlines
    #.limit(100)
    .write.mode('overwrite')
    .saveAsTable(f'{db_name}.{table_name}')
)

In [0]:
%sql
SELECT COUNT(*) FROM airline.airline_part_00000 -- 645.918

#### Checando detalhes

In [0]:
warehouse_uri = 'dbfs:/user/hive/warehouse/'
display(dbutils.fs.ls(warehouse_uri))

In [0]:
warehouse_airline_uri = 'dbfs:/user/hive/warehouse/airline.db/'
display(dbutils.fs.ls(warehouse_airline_uri))

In [0]:
airline_part_0000_uri = 'dbfs:/user/hive/warehouse/airline.db/airline_part_00000/'
display(dbutils.fs.ls(airline_part_0000_uri))

In [0]:
airline_delta_logs_uri = 'dbfs:/user/hive/warehouse/airline.db/airline_part_00000/_delta_log/'
display(dbutils.fs.ls(airline_delta_logs_uri))

#### Verificando se está em Delta

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, airline_part_0000_uri)

df_detail = deltaTable.detail()
df_detail.display()

In [0]:
DeltaTable.isDeltaTable(spark, airline_part_0000_uri)